In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data3500-10/data3500_10
/kaggle/input/mainnnn/mainediteddatasettt.csv


In [25]:
data=pd.read_csv('/kaggle/input/data3500-10/data3500_10')
data=data.drop(['Unnamed: 0',' Novel',' Young adult literature',' Suspense',' Speculative fiction',' Science Fiction'],1)
#data = data.loc[np.random.choice(data.index, size=3500)]
print(data.shape)
data.head()
#data.to_csv('data3500_5')

(3500, 8)


In [4]:
import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [5]:

def cleanbasic(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned


def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

In [6]:
data['Summary'] = data['Summary'].str.lower()
data['Summary'] = data['Summary'].apply(cleanbasic)
data['Summary'] = data['Summary'].apply(cleanPunc)
data['Summary'] = data['Summary'].apply(keepAlpha)
data.head()

,Title,Author/Publisher,Summary,Children's literature,Crime Fiction,Fantasy,Fiction
0,Shaman's Crossing,Robin Hobb,young nevare burvelle pronounced nuh vair is t...,0,0,1,0
1,The Fashion in Shrouds,Margery Allingham,campion asks his sister fashion designer valen...,0,1,1,1
2,River of Smoke,Amitav Ghosh,the promotional text refers to the storyline w...,0,0,0,0
3,Heaven Cent,Piers Anthony,the story begins with prince dolph lying in hi...,0,0,1,1
4,Toby Alone,TimothÃ©e de Fombelle,a year boy named toby lolness who is just one...,0,0,0,0


In [7]:
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

data['Summary'] = data['Summary'].apply(removeStopWords)
#data=data.drop('Unnamed: 0',1)
data.head()

,Title,Author/Publisher,Summary,Children's literature,Crime Fiction,Fantasy,Fiction
0,Shaman's Crossing,Robin Hobb,young nevare burvelle pronounced nuh vair se...,0,0,1,0
1,The Fashion in Shrouds,Margery Allingham,campion asks sister fashion designer valentin...,0,1,1,1
2,River of Smoke,Amitav Ghosh,promotional text refers storyline summar...,0,0,0,0
3,Heaven Cent,Piers Anthony,story begins prince dolph lying sister ivy...,0,0,1,1
4,Toby Alone,TimothÃ©e de Fombelle,year boy named toby lolness half milli...,0,0,0,0


In [8]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

data['Summary'] = data['Summary'].apply(stemming)
data.head()

,Title,Author/Publisher,Summary,Children's literature,Crime Fiction,Fantasy,Fiction
0,Shaman's Crossing,Robin Hobb,young nevar burvell pronounc nuh vair second s...,0,0,1,0
1,The Fashion in Shrouds,Margery Allingham,campion ask sister fashion design valentin fer...,0,1,1,1
2,River of Smoke,Amitav Ghosh,promot text refer storylin summar follow incid...,0,0,0,0
3,Heaven Cent,Piers Anthony,stori begin princ dolph lie sister ivi bed wat...,0,0,1,1
4,Toby Alone,TimothÃ©e de Fombelle,year boy name tobi lol half millimet tall live...,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, random_state=42, test_size=0.20, shuffle=True)


In [10]:
train_text = train['Summary']
test_text = test['Summary']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode')

In [12]:
x_train = vectorizer.transform(train_text)
y_train = train.drop(labels = ['Title','Author/Publisher','Summary'], axis=1)
y_train = y_train.apply(pd.to_numeric,errors='ignore')

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels = ['Title','Author/Publisher','Summary'], axis=1)
y_test = y_test.apply(pd.to_numeric,errors='ignore')


In [13]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.5671428571428572




In [15]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(LogisticRegression())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.41285714285714287




In [16]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

In [17]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
classifier = ClassifierChain(GaussianNB())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

accuracy_score(y_test,predictions)

0.5671428571428572

In [18]:

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())

# Training logistic regression model on train data
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.3914285714285714




In [19]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.metrics import accuracy_score

In [20]:
%%time

# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.4685714285714286


CPU times: user 6min 37s, sys: 7.17 s, total: 6min 44s
Wall time: 3min 35s


In [21]:
%%time

# initialize label powerset multi-label classifier
classifier = LabelPowerset(GaussianNB())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.5885714285714285


CPU times: user 21.8 s, sys: 19.3 s, total: 41 s
Wall time: 41.1 s
